In [2]:
from functools import reduce
from py_ecc import bn128
from py_ecc.bn128 import G1, G2, bn128_curve, curve_order, multiply, eq, neg, field_modulus, pairing
from sage.all import *

F = GF(field_modulus)
F_x.<x> = F[] # polynomial over finite field

A = Matrix(F, [
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0]
])

B = Matrix(F, [
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1]
])

C = Matrix(F,[
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [0,1,0,0,0,0,0,0]
])

# x*2 + x + 3

x =  F.random_element()
y =  F.random_element()
z =  F.random_element()
u =  F.random_element()
l = 2 # public variables index


out = x * y * z * u
v1 = x*y
v2 = z*u

# create the witness vector
witness = vector(F, [1, out, x, y, z, u, v1, v2])


assert Matrix(C * witness) == Matrix(A * witness).elementwise_product(Matrix(B * witness))

def bn_add(A, B):
    return bn128_curve.add(A, B)

def trusted_setup(n, t_x, U_polys, V_polys, W_polys):
    encrypted_G1 = list()
    encrypted_G2 = list()
    t_x_enc = list()

    tau = F.random_element()
    pow_of_tau = 1  # tau ** 0


    alpha = F.random_element()
    beta = F.random_element()
    gama = F.random_element()
    delta = F.random_element()
    gama_inv = inverse_mod(gama, field_modulus)
    delta_inv = inverse_mod(delta, field_modulus)

    Alpha = multiply(G1, int(alpha))
    Beta_1 = multiply(G1, int(beta))
    Beta_2 = multiply(G2, int(beta))
    Gama = multiply(G2, int(gama))
    Delta_1 = multiply(G1, int(delta))
    Delta_2 = multiply(G2, int(delta))
    t_x_eval = t_x(tau)

    for i in range(0, n):
        temp = multiply(G1, int(pow_of_tau))
        encrypted_G1.append(temp)
        encrypted_G2.append(multiply(G2, int(pow_of_tau)))

        if i<= n-2:
            t_x_enc.append(multiply(temp, int(t_x_eval)))
        pow_of_tau = pow_of_tau*tau
        


    betaU = [multiply(p, int(beta)) for p in encrypt_polynomials(U_polys, encrypted_G1)]
    alphaV =[multiply(p, int(alpha)) for p in encrypt_polynomials(V_polys, encrypted_G1)]
    w = encrypt_polynomials(W_polys, encrypted_G1)

    betaU_pub = betaU[:l]
    betaU_priv = betaU[:l]

    alphaV_pub = alphaV[:l]
    alphaV_priv = alphaV[:l]

    w_pub = w[:l]
    w_priv = w[:l]

    def add_(a, b, c):
        return bn_add(a, bn_add(b, c))
    pow_tau_C_pub = list(map(add_, betaU_pub, alphaV_pub, w_pub))
    pow_tau_C_priv = list(map(add_, betaU_priv, alphaV_priv, w_priv))

    def mult_with_gama_inv(p):
        return multiply(p, gama_inv)
    
    def mult_with_delta_inv(p):
        return multiply(p, delta_inv)

    pow_tau_C_pub = list(map(mult_with_gama_inv, pow_tau_C_pub))
    pow_tau_C_priv = list(map(mult_with_delta_inv, pow_tau_C_priv))

    return encrypted_G1, encrypted_G2, t_x_enc, Alpha, Beta_1, Beta_2, Gama, Delta_1, Delta_2, pow_tau_C_pub, pow_tau_C_priv

def generate_target_polynomial(n):
    # Create a polynomial of the form (x - 1)(x - 2)(x - 3)...(x - n)
    x = var('x')
    res = F_x(prod([(x - i) for i in range(1, n+1)]))
    for i in range(1, n+1):
        assert res(i) == 0, "Invalid target polynom"
    return res

def interpolate_polynomials(matrix):
    result = list()
    for column in matrix.columns():
        polynomial = F_x.lagrange_polynomial([(index+1, values) for index, values in enumerate(column)])
        result.append(polynomial)
    return result

def inner_product_polynomials_with_witness(polys, witness):
    def mul(x, y): return x * y
    def sum(x, y): return x + y
    return reduce(sum, map(mul, polys, witness))

def inner_product_enc_polynomials_with_witness(polys, witness):
    def mul(x, y): return multiply(x, int(y))
    return reduce(bn_add, map(mul, polys, witness))

def encrypt_polynomials(polynomials, encrypted_points):
    encrypted_polynomials = [None] * len(polynomials)
    for i, poly in enumerate(polynomials):
        if poly != 0:
            for j, coeff in enumerate(poly.coefficients(sparse=False)):
                encrypted_polynomials[i] = bn_add(encrypted_polynomials[i], multiply(encrypted_points[j], int(coeff)))
    return encrypted_polynomials

def encrypt_hx_tx(h_x, t_x_enc):
    h_x_enc = list()
    for index, coeff in enumerate(h_x.coefficients(sparse=False)):
        h_x_enc.append(multiply(t_x_enc[index], int(coeff)))
    return reduce(bn_add, h_x_enc)

public_witness = witness[:l]
private_witness = witness[l:]

t_x = generate_target_polynomial(len(A.rows()))
U_polys = interpolate_polynomials(A)
V_polys = interpolate_polynomials(B)
W_polys = interpolate_polynomials(C)

(encrypted_G1, encrypted_G2, t_x_enc, Alpha, Beta_1, Beta_2, Gama, Delta_1, Delta_2, pow_of_tau_public, pow_of_tau_private) = trusted_setup(len(A.rows()), t_x, U_polys, V_polys, W_polys)

r = F.random_element()
s = F.random_element()

poly1 = inner_product_polynomials_with_witness(U_polys, witness)
poly2 = inner_product_polynomials_with_witness(V_polys, witness)
poly3 = inner_product_polynomials_with_witness(W_polys, witness)  

h_x, remainder = ((poly1 * poly2) - poly3).quo_rem(t_x)
assert remainder == 0, "Division has a ramainder"

hx_tx_enc = encrypt_hx_tx(h_x, t_x_enc)

A_1 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(U_polys, encrypted_G1), witness)
B_1 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(V_polys, encrypted_G1), witness)
B_2 = inner_product_enc_polynomials_with_witness(encrypt_polynomials(V_polys, encrypted_G2), witness)
C_1_private = inner_product_enc_polynomials_with_witness(pow_of_tau_private, private_witness)
C_1_public = inner_product_enc_polynomials_with_witness(pow_of_tau_public, public_witness)


R_1 = multiply(Delta_1, r)
S_1 = multiply(Delta_1, s)
S_2 = multiply(Delta_2, s)

A_1 = bn_add(bn_add(A_1, Alpha), R_1)
B_1 = bn_add(bn_add(B_1, Beta_1), S_1)
B_2 = bn_add(bn_add(B_2, Beta_2), S_2)
C_1_public = bn_add(C_1_public, hx_tx_enc)
C_1_private = bn_add(C_1_private, hx_tx_enc)

s_A_1 = multiply(A_1, s)
r_B_1 = multiply(B_1, r)
rs_Delta_1 = multiply(Delta_1)
C_1 = bn_add(bn_add(C_1_private, C_1_public), bn_add(bn_add(s_A_1, r_B_1), neg(rs_Delta_1)))

pairing_1 = pairing(B_2, A_1) 
pairing_2 = pairing(G2, C_1)

assert pairing_1 == pairing_2, "Fail"


AssertionError: Fail